In [49]:
import requests
from bs4 import BeautifulSoup
import time
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import japanize_matplotlib


In [50]:
# データベースの準備
dbname = '最終課題.db'
conn = sqlite3.connect(dbname)
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS properties') # 初期化用
cur.execute('''
    CREATE TABLE properties (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT, station TEXT, price INTEGER, age INTEGER, floor_plan TEXT
    )
''')

# スクレイピング実行
def get_suumo_data():
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)"}
    base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1"
    
    for page in range(1, 4):
        print(f"--- Page {page} を取得中... ---")
        url = f"{base_url}&page={page}"
        res = requests.get(url, headers=headers)
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text, 'html.parser')
        items = soup.find_all("div", class_="cassetteitem")
        
        data_list = []
        for item in items:
            try:
                # 物件名の取得（存在チェック付き）
                title_elem = item.find("div", class_="cassetteitem_content-title")
                name = title_elem.text.strip() if title_elem else "不明"
                
                # 駅情報の取得（エラー箇所：存在チェックを追加）
                station_elem = item.find("div", class_="cassetteitem_detail-col1")
                station = station_elem.text.strip() if station_elem else "不明"
                
                # 部屋情報の取得
                tbody = item.find("table", class_="cassetteitem_other")
                if tbody:
                    for tr in tbody.find("tbody").find_all("tr"):
                        try:
                            price_elem = tr.find_all("td")[3].find("li")
                            if price_elem:
                                price_text = price_elem.text.strip()
                                price = int(float(price_text.replace("万円", "")) * 10000)
                                floor_plan = tr.find_all("td")[2].text.strip()
                                data_list.append((name, station, price, 0, floor_plan))
                        except:
                            continue # 部屋単位のエラーはスキップ
            except Exception as e:
                print(f"物件スキップ: {e}")
                continue
                    
        if data_list:
            cur.executemany("INSERT INTO properties (name, station, price, age, floor_plan) VALUES (?, ?, ?, ?, ?)", data_list)
            conn.commit()
            print(f" -> {len(data_list)} 件保存しました")
        
        time.sleep(3) # 負荷対策
    print("完了！ 最終課題.db に保存しました。")

conn.close()

In [51]:
conn = sqlite3.connect('最終課題.db')
df = pd.read_sql("SELECT * FROM properties", conn)
conn.close()

if not df.empty:
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, x='floor_plan', y='price')
    plt.title('間取りごとの家賃分布（SUUMO調査データ）')
    plt.xlabel('間取り')
    plt.ylabel('家賃（円）')
    plt.show()
else:
    print("データが空です。スクレイピングをやり直してください。")

データが空です。スクレイピングをやり直してください。
